# EZyRB Tutorial 3
## Use NNsPOD to help with POD

In this tutorial we show how to set up and use the NNsPOD class in order to make all data align, allowing the use of POD.

To do this we will show a simple example where the data is a moving gaussian wave.

the first step is to import necessary packages

In [1]:
import numpy as np
import matplotlib
import torch
import torch.nn as nn
from ezyrb.nnspod import NNsPOD
from ezyrb import Database
from ezyrb import POD
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

## 1d Data

Now we make the data we will use. We make a simple gaussian function and populate the space, snapshots, and parameters of the database

In [2]:
n_params = 15
params = np.linspace(0.5, 4.5, n_params).reshape(-1, 1) # actually the time steps
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
def wave(t, res=256):
    x = np.linspace(0, 5, res)
    return  x, gaussian(x, t, 0.1)

db = np.array([wave(t)[1] for t in params])
db_array =  np.array([wave(t)[1] for t in params])
space = wave(0)[0]
space_array = np.array([wave(t)[0] for t in params])

database = Database(space = space_array, snapshots = db_array, parameters = params)


Here we make a NNsPOD class, the only value to pass in is where you want to save the interpnet, or where you want to load it from. This is especially usefull with 2d data where training can take hours depending on the size of the dataset.

In [3]:
ref_data = 5
NNsPOD_tutorial = NNsPOD(None, [20,20], nn.Sigmoid(), [0.000001], ref_data,
                        shift_layers = [20,20,20], shift_function = nn.Tanh(), shift_stop_training =[10000, 0.00001]
                        
)

Now we train the interpnet. the data to pass in is the reference database, the shape of the layers of the NN, the trainnig function the stop training value, which can be a float(loss value to stop at), int(epoch to stop at), or both(will stop at whichever is reached first).The loss function(MSE by default). and whether you would like to retrain NN or load a saved NN. If you choose to retrain the loss value at each epoch will be printed out.

In [4]:

NNsPOD_tutorial.train_interpnet(database[ref_data],retrain  = False, frequency_print = 5, interp_file = "interpnet1d.pth")

interpnet1d.pth
loaded interpnet


Now we can graph the original reference data as well as the data we get from the interpNet after feeding it 1000 positional datapoints. The large points are the original data points, and the small points are ones created by the interpnet. It should be clear that the interpnet is able to accuratly replicate the gaussian

In [5]:
plt.plot(database[ref_data].space, database[ref_data].snapshots, "o")
xi = np.linspace(0,5,1000).reshape(-1,1)
yi = NNsPOD_tutorial.interp_net.predict(xi)
plt.plot(xi,yi, ".")
plt.show()

Now we train the shiftnet on all data besides the reference. to do this you must pass in the database at the value, the shape of the NN, the training function, the stop training value, the reference database, and if you would like the data to be preshifted. For the shiftnet it can be useful to put a loss value and epoch value to stop at, as there is a minimum level the loss value can reach, and if you put a lower value the neural net will not stop.

Training the shiftnet also prints out the loss value at every epoch

In [6]:
i = 0
while i < 10:
    x_new  = NNsPOD_tutorial.train_shiftnet(database[i], database[ref_data], preshift = True, frequency_print = 5, learning_rate = 0.0001) 
    db = database[i]   
    plt.plot(db.space, db.snapshots, "go")
    plt.plot(x_new, db.snapshots.reshape(-1,1), ".")
    i+=1
    if i == ref_data:
        i +=1

0.021576717495918274
0.018197430297732353
0.014862452633678913
0.011553604155778885
0.00839116983115673
0.005527989007532597
0.0031261462718248367
0.0013361244928091764
0.0002760965726338327
0.009425859898328781
0.005659420974552631
0.0026965439319610596
0.0007226068992167711
1.6661051631672308e-05
0.0007273565279319882
0.0027560549788177013
0.005670929327607155
0.008676070719957352
0.010716329328715801
0.010876988060772419
0.008965139277279377
0.005723173264414072
0.002428542822599411
0.00032989634200930595
0.0002644015767145902
0.0024630895350128412
0.006529766134917736
0.011595336720347404
0.01659214496612549
0.02053970657289028
0.02274174429476261
0.022853480651974678
0.020864220336079597
0.017085248604416847
0.012163663282990456
0.007046045735478401
0.002808806486427784
0.0003732281329575926
0.04303792864084244
0.03818920627236366
0.03288600593805313
0.027048692107200623
0.020863473415374756
0.014669893309473991
0.008941804990172386
0.004238499328494072
0.0011151344515383244
1.976

Here we plot and show all the data. The original positions is represented by green circles, the reference data is the blue plusmarks, and the different shifted data is represented by the smaller dots. It should be clear that all of the data has been moved to allign with the reference data

In [7]:
plt.plot(database[0].space, database[ref_data].snapshots, "b+")
plt.show()

Here we use the NNsPOD fit function, and we get get the modes and singular values

In [8]:
pod = NNsPOD_tutorial.fit(interp_file = "interpnet1d.pth",
                          db = database)
print(pod.modes, pod.singular_values)

loaded interpnet
0.0011340980418026447
0.037661511451005936
0.0011428085854277015
0.018658315762877464
0.010687826201319695
0.005290625151246786
0.022084934636950493
0.006550148129463196
0.004208308644592762
0.01799725741147995
0.00940329022705555
0.00030273222364485264
0.012823530472815037
0.018274104222655296
0.008066706359386444
1.0090350770042278e-05
0.0055536674335598946
0.005161964800208807
0.015556512400507927
0.010334138758480549
0.008668169379234314
0.015472044236958027
0.01992838829755783
0.0002611768722999841
0.005396030843257904
0.004713937174528837
0.015291141346096992
0.023489514365792274
0.009426423348486423
0.002368635730817914
0.06547225266695023
0.020965032279491425
0.02389248088002205
0.027566159144043922
0.003045934485271573
0.031973134726285934
0.01897011697292328
0.0004547239514067769
0.01814550906419754
0.017512764781713486
0.0004336966958362609
0.013621974736452103
0.02712256833910942
0.016880594193935394
0.0003777860547415912
0.02320970594882965
0.0007515900651

In [9]:
n_params = 15
params = np.linspace(0.5, 4.5, n_params).reshape(-1, 1) # actually the time steps
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
def wave(t, res=256):
    x = np.linspace(0, 5, res)
    return  x, gaussian(x, 2, 0.1)

db = np.array([wave(t)[1] for t in params])
db_array =  np.array([wave(t)[1] for t in params])
space = wave(0)[0]
space_array = np.array([wave(t)[0] for t in params])

database = Database(space = space_array, snapshots = db_array, parameters = params)

pod = POD()

pod.fit(database.snapshots)
print(pod.modes, pod.singular_values)



[[-2.58198890e-01  5.59788262e-01  7.87382013e-01 -6.19866111e-16
   2.68173182e-16 -5.67483059e-17  1.74610172e-17 -1.30957629e-17
   0.00000000e+00  8.73050861e-18  8.73050861e-18 -3.38307208e-17
   3.49220344e-17  3.27394073e-17 -3.49220344e-17]
 [-2.58198890e-01 -1.28910610e-01  6.98006745e-03  9.57427108e-01
   1.48987716e-17 -1.32856031e-16  1.33295622e-16  2.25883973e-16
  -2.86503678e-17 -3.35625782e-17 -1.53950992e-17  3.31617815e-17
  -2.01522788e-17  4.57926075e-18 -1.24573224e-17]
 [-2.58198890e-01 -1.28910610e-01  6.98006745e-03 -8.70388280e-02
  -7.44298646e-17  9.49214452e-01 -2.34327101e-02  8.44009971e-02
  -2.02537247e-02 -2.91328554e-17 -8.91115352e-17 -6.85128563e-18
   3.65744161e-18 -1.55735773e-17 -6.07114327e-19]
 [-2.58198890e-01 -1.28910610e-01  6.98006745e-03 -8.70388280e-02
   3.41662710e-17 -1.39215195e-01  6.76185044e-01  6.39480270e-01
  -1.53456212e-01 -1.46289570e-16 -6.60239786e-16  3.48154415e-17
   1.92611922e-17 -3.06586380e-17  9.90641208e-18]
 [-2

## 2d Data

Now we do the same but with 2d data and implement some basic functions to help with shaping the data

In [20]:
def reshape2dto1d(x, y):
    x = x.reshape(-1,1)
    y = y.reshape(-1,1)
    coords = np.concatenate((x, y), axis = 1)
    coords = np.array(coords).reshape(-1,2)
    
    return coords

def reshape1dto2d(snapshots):
        return snapshots.reshape(int(np.sqrt(len(snapshots))), int(np.sqrt(len(snapshots))))


Here we create the 2d gaussian and populate the database

In [21]:

n_params = 10
params = np.linspace(0.5, 4.5, n_params).reshape(-1, 1) # actually the time steps

def gaussian(x, mu, sig):
    print(mu)
    gaussx, gaussy = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))).T
    return gaussx * gaussy
def wave(t, res=256):
    x = np.linspace(0, 5, res)
    return  x, gaussian(x, t, 0.1)
def wave2D(t, res=256):
    x = np.linspace(0, 5, res)
    y = np.linspace(0, 5, res)
    gridx, gridy = np.meshgrid(x, y)
    gridx, gridy = gridx.reshape(-1,1), gridy.reshape(-1,1)
    wave = gaussian(np.hstack([gridx, gridy]), t*np.array([1, 1]), 0.1)
    return  gridx, gridy, wave
db = np.array([wave2D(t)[2] for t in params])
db_array = db.reshape(n_params, -1, 1)
gridx, gridy = wave2D(0)[0 :2]
space = reshape2dto1d(gridx,gridy)
space_array = np.array([space.copy() for t in params])

database = Database(space = space_array, snapshots = db_array, parameters = params)


[0.5 0.5]
[0.94444444 0.94444444]
[1.38888889 1.38888889]
[1.83333333 1.83333333]
[2.27777778 2.27777778]
[2.72222222 2.72222222]
[3.16666667 3.16666667]
[3.61111111 3.61111111]
[4.05555556 4.05555556]
[4.5 4.5]
[0 0]


In [22]:
ref_point = 5
NNsPOD_tutorial = NNsPOD(interp_loss = None, interp_layers = [40,40], interp_function = nn.Sigmoid(), interp_stop_training = [10000000,0.000001], ref_point = ref_point,
                         shift_layers = [20,20,20], shift_function = nn.PReLU(), shift_stop_training = [0.001])

Here we put the value for the reference data and train the interpnet. If you want to change this it will need to retrain the interpnet, which can take a long time for 2d data, especially is the loss value is very low.

In [23]:

ref_data = 5
NNsPOD_tutorial.train_interpnet(database[ref_data], retrain  = False, frequency_print = 5, interp_file = "interpnet2d.pth")

interpnet2d.pth
loaded interpnet


Here we graph the reference data

In [24]:
x = np.linspace(0, 5, 256)
y = np.linspace(0, 5, 256)
gridx, gridy = np.meshgrid(x, y)
        
plt.pcolor(gridx,gridy,database[ref_data].snapshots.reshape(256, 256))
plt.show()

Now we graph the interpolated data. it should be visisble that we get the same function, but with better resolution

In [25]:
res = 1000
x = np.linspace(0, 5, res)
y = np.linspace(0, 5, res)
gridx, gridy = np.meshgrid(x, y)
input = NNsPOD_tutorial.reshape2dto1d(gridx, gridy)
output = NNsPOD_tutorial.interp_net.predict(input)

toshow = NNsPOD_tutorial.reshape1dto2d(output)
plt.pcolor(gridx,gridy,toshow)
plt.show()

Now we use the shiftnet and graph the shifted data. For each parameter we first graph the refrence data, then the input data, then it will take some time to shift it, and finally it will graph the shifted data. The loss value at every epoch will be printed out as well.

In [26]:
i = 0
x = np.linspace(0, 5, 256)
y = np.linspace(0, 5, 256)
gridx, gridy = np.meshgrid(x, y)
while i < database.parameters.shape[0]:
    db = database[i]
    plt.pcolor(gridx,gridy,database[ref_data].snapshots.reshape(256, 256))
    plt.show()
    plt.pcolor(gridx,gridy,database[i].snapshots.reshape(256, 256))
    plt.show()
    x_new = NNsPOD_tutorial.train_shiftnet(database[i],  database[ref_data], preshift = True, frequency_print = 5)
    x, y = np.hsplit(x_new, 2)
    x = NNsPOD_tutorial.reshape1dto2d(x)
    y = NNsPOD_tutorial.reshape1dto2d(y)
    snapshots = NNsPOD_tutorial.reshape1dto2d(db.snapshots.reshape(-1,1))
    plt.pcolor(x,y,snapshots)
    plt.show()
    res = 256
    i+=1
    if i == ref_data:
        i +=1



0.0018307577120140195
0.0016863499768078327
0.0015271008014678955
0.0013487492688000202


C:\Users\mrowk\AppData\Local\Temp\ipykernel_20536\800095035.py:16: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  plt.pcolor(x,y,snapshots)


0.002387014217674732
0.0023464937694370747
0.002300079446285963
0.0022443446796387434
0.0021787662990391254
0.0021018616389483213
0.0020114330109208822
0.001906601944938302
0.0017864166293293238
0.0016403829213231802
0.001478851423598826
0.0013043213402852416


KeyboardInterrupt: 

In [27]:
pod = NNsPOD_tutorial.fit(interp_file = "interpnet2d.pth",
                          db = database)
print(pod.modes, pod.singular_values)

0.001274904003366828
0.001253377995453775
0.0012363020796328783
0.0012356039369478822
0.0012348229065537453
0.0012340829707682133
0.0012333827326074243
0.0012327131116762757
0.0012320721289142966
0.0012314565246924758
0.001230863039381802
0.0012302907416597009
0.0012297385837882757
0.0012292059836909175
0.001228693057782948
0.0012281996896490455
0.0012277251807972789
0.0012272682506591082


KeyboardInterrupt: 